# Local Density vs Causal Norm

**Goal:** Determine if the "void" at the core is a true void (low density) or just low count.

**Method:**
- Use precomputed 32k×32k distance matrix
- For each token: compute local density via k-nearest-neighbor distances
- For each token: compute causal norm ||γᵢ||_M
- Plot density vs norm to see radial density profile

**Key question:** Are core tokens (small norm) actually dense or sparse?

**Inputs:**
- `data/vectors/distances_causal_32000_full.npy` - Distance matrix
- `data/vectors/distances_causal_32000.pt` - Token indices
- Model for token embeddings + metric tensor

**Expected runtime:** ~1-2 minutes

## Configuration

In [24]:
# Input files
INPUT_DISTANCES = '../data/vectors/distances_causal_32000_full.npy'
INPUT_METADATA = '../data/vectors/distances_causal_32000.pt'
METRIC_TENSOR_PATH = '../data/vectors/causal_metric_tensor_qwen3_4b.pt'

# Model configuration
MODEL_NAME = 'Qwen/Qwen3-4B-Instruct-2507'
DEVICE = 'cpu'  # Local analysis, no GPU needed

# Density parameters
K_NEIGHBORS = 20  # Number of nearest neighbors for density estimate

print(f"Configuration:")
print(f"  Distance matrix: {INPUT_DISTANCES}")
print(f"  k-NN: {K_NEIGHBORS} neighbors")
print(f"  Model: {MODEL_NAME}")

Configuration:
  Distance matrix: ../data/vectors/distances_causal_32000_full.npy
  k-NN: 20 neighbors
  Model: Qwen/Qwen3-4B-Instruct-2507


## Setup

In [25]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM

print("✓ Imports complete")

✓ Imports complete


## Load Distance Matrix

In [26]:
print(f"Loading distance matrix from {INPUT_DISTANCES}...")
distances = np.load(INPUT_DISTANCES)

N = distances.shape[0]

print(f"✓ Loaded distance matrix")
print(f"  Shape: {distances.shape}")
print(f"  N tokens: {N:,}")
print(f"  Memory: {distances.nbytes / 1e9:.2f} GB")

Loading distance matrix from ../data/vectors/distances_causal_32000_full.npy...
✓ Loaded distance matrix
  Shape: (32000, 32000)
  N tokens: 32,000
  Memory: 4.10 GB


## Load Token Indices

In [27]:
print(f"\nLoading token indices from {INPUT_METADATA}...")
metadata = torch.load(INPUT_METADATA, weights_only=False)

token_indices = metadata['token_indices'].numpy()

print(f"✓ Loaded token indices")
print(f"  Mapping {N:,} rows/cols → vocabulary indices")
print(f"  Range: [{token_indices.min()}, {token_indices.max()}]")


Loading token indices from ../data/vectors/distances_causal_32000.pt...
✓ Loaded token indices
  Mapping 32,000 rows/cols → vocabulary indices
  Range: [5, 151930]


## Compute k-NN Density

For each token, density = 1 / (mean distance to k nearest neighbors)

High density = neighbors are close  
Low density = neighbors are far away

In [28]:
print(f"\nComputing k-NN density (k={K_NEIGHBORS})...")

densities = np.zeros(N)

for i in range(N):
    # Get distances from token i to all others
    dists_i = distances[i]
    
    # Sort to find k nearest (excluding self at distance 0)
    # Use partition for efficiency (O(N) instead of O(N log N))
    k_nearest_dists = np.partition(dists_i, K_NEIGHBORS)[:K_NEIGHBORS+1]
    
    # Remove self-distance (should be 0)
    k_nearest_dists = k_nearest_dists[k_nearest_dists > 0][:K_NEIGHBORS]
    
    # Density = 1 / mean distance
    mean_dist = k_nearest_dists.mean()
    densities[i] = 1.0 / mean_dist if mean_dist > 0 else 0

print(f"✓ Computed densities")
print(f"\nDensity statistics:")
print(f"  Min: {densities.min():.6f}")
print(f"  Max: {densities.max():.6f}")
print(f"  Mean: {densities.mean():.6f}")
print(f"  Median: {np.median(densities):.6f}")


Computing k-NN density (k=20)...


/var/folders/k5/44vd1ct56xj4y9h7x213kvjr0000gn/T/ipykernel_94387/215823279.py:17: RuntimeWarning: Mean of empty slice.
  mean_dist = k_nearest_dists.mean()


✓ Computed densities

Density statistics:
  Min: 0.000000
  Max: 2413.602051
  Mean: 0.595077
  Median: 0.019493


## Convert to Volumetric Density

k-NN density measures "how close are neighbors" but doesn't account for high-dimensional geometry.

True volumetric density = points per unit hypervolume

For k neighbors at mean distance r:
- Volume of d-ball: V_d(r) ∝ r^d
- Volumetric density: ρ_vol ≈ k / V_d(r)

We use effective dimensionality d ≈ 1333 (from participation ratio analysis).

In [29]:
from scipy.special import gamma as gamma_func

# Effective dimensionality from participation ratio (notebook 04.1)
D_EFF = 1333

print(f"\nComputing volumetric density...")
print(f"  Effective dimensionality: {D_EFF}")

# For each token, we have k-NN density = 1 / mean_distance
# We need to recover mean_distance and compute volume

mean_distances = np.zeros(N)
for i in range(N):
    if densities[i] > 0:
        mean_distances[i] = 1.0 / densities[i]
    else:
        mean_distances[i] = np.nan

# For numerical stability, compute log(volumetric_density) instead
# log(ρ_vol) = log(k) - log(V_d(r))
#            = log(k) - [(d/2)*log(π) - log(Γ(d/2+1)) + d*log(r)]

log_volumetric_densities = np.zeros(N)

for i in range(N):
    if mean_distances[i] > 0 and not np.isnan(mean_distances[i]):
        r = mean_distances[i]
        
        # log(V_d(r)) = (d/2)*log(π) - log(Γ(d/2+1)) + d*log(r)
        log_vol = (D_EFF/2) * np.log(np.pi) - np.log(gamma_func(D_EFF/2 + 1)) + D_EFF * np.log(r)
        
        # log(ρ_vol) = log(k) - log(V_d(r))
        log_volumetric_densities[i] = np.log(K_NEIGHBORS) - log_vol
    else:
        log_volumetric_densities[i] = -np.inf

# Convert back to linear scale for valid entries
volumetric_densities = np.exp(log_volumetric_densities)

# Filter for statistics
valid_mask = np.isfinite(volumetric_densities) & (volumetric_densities > 0)
n_valid = valid_mask.sum()

print(f"\n✓ Computed volumetric densities")
print(f"  Valid entries: {n_valid:,} / {N:,}")

if n_valid > 0:
    print(f"\nVolumetric density statistics:")
    print(f"  Min: {volumetric_densities[valid_mask].min():.6e} tokens/logometer^{D_EFF}")
    print(f"  Max: {volumetric_densities[valid_mask].max():.6e} tokens/logometer^{D_EFF}")
    print(f"  Mean: {volumetric_densities[valid_mask].mean():.6e} tokens/logometer^{D_EFF}")
    print(f"  Median: {np.median(volumetric_densities[valid_mask]):.6e} tokens/logometer^{D_EFF}")
    
    print(f"\nLog-space statistics (for reference):")
    print(f"  Log min: {log_volumetric_densities[valid_mask].min():.2f}")
    print(f"  Log max: {log_volumetric_densities[valid_mask].max():.2f}")
    print(f"  Log mean: {log_volumetric_densities[valid_mask].mean():.2f}")
else:
    print("\n⚠️  No valid volumetric densities computed!")
    print("  This suggests numerical overflow/underflow issues.")
    print("  Volumes at r^1333 are astronomically large!")


Computing volumetric density...
  Effective dimensionality: 1333

✓ Computed volumetric densities
  Valid entries: 0 / 32,000

⚠️  No valid volumetric densities computed!
  This suggests numerical overflow/underflow issues.
  Volumes at r^1333 are astronomically large!


## Volumetric Density vs Causal Norm

This shows true points per unit hypervolume, accounting for high-dimensional geometry.

In [30]:
# We need causal norms first - load them if not already computed
# This cell assumes causal_norms is available from later cells
# For now, we'll compute them here to keep the flow logical

print(f"\nLoading model and computing causal norms...")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,
    device_map=DEVICE,
)

gamma = model.lm_head.weight.data
del model

metric_data = torch.load(METRIC_TENSOR_PATH, weights_only=False)
M = metric_data['M']

sampled_embeddings = gamma[token_indices]
M_gamma = sampled_embeddings @ M.to(DEVICE)
causal_norms = torch.sqrt((sampled_embeddings * M_gamma).sum(dim=-1)).numpy()

print(f"✓ Computed causal norms for volumetric plot")

# Now plot volumetric density vs norm
fig, ax = plt.subplots(figsize=(12, 6))

# Filter out zeros for log scale
valid_mask = volumetric_densities > 0

scatter = ax.scatter(
    causal_norms[valid_mask],
    volumetric_densities[valid_mask],
    c=volumetric_densities[valid_mask],
    cmap='viridis',
    alpha=0.5,
    s=10,
    rasterized=True,
    norm=plt.matplotlib.colors.LogNorm()  # Log color scale
)

plt.colorbar(scatter, ax=ax, label=f'Volumetric Density (tokens/logometer^{D_EFF})')

ax.set_xlabel('Causal Norm (logometers)', fontsize=12)
ax.set_ylabel(f'Volumetric Density (tokens/logometer^{D_EFF})', fontsize=12)
ax.set_yscale('log')
ax.set_title(f'Volumetric Density vs Causal Norm (d={D_EFF})', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n💡 This accounts for high-dimensional geometry:")
print(f"  - Volume of d-ball grows as r^{D_EFF}")
print(f"  - If tokens were uniformly distributed, volumetric density would be constant")
print(f"  - Power-law decay in volumetric density = non-uniform distribution")


Loading model and computing causal norms...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✓ Computed causal norms for volumetric plot


ValueError: Invalid vmin or vmax

Error in callback <function _draw_all_if_interactive at 0x123ea1440> (for post_execute), with arguments args (),kwargs {}:


ValueError: Invalid vmin or vmax

ValueError: Invalid vmin or vmax

<Figure size 1200x600 with 2 Axes>

## Load Model and Compute Causal Norms

In [ ]:
print(f"\nLoading model to extract token embeddings...")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,
    device_map=DEVICE,
)

# Extract unembedding matrix γ
gamma = model.lm_head.weight.data  # [vocab_size, hidden_dim]

print(f"✓ Extracted unembedding matrix")
print(f"  Shape: {gamma.shape}")

# Free model memory
del model
print(f"✓ Freed model memory")

In [ ]:
print(f"\nLoading metric tensor from {METRIC_TENSOR_PATH}...")
metric_data = torch.load(METRIC_TENSOR_PATH, weights_only=False)

M = metric_data['M']  # [hidden_dim, hidden_dim]

print(f"✓ Loaded metric tensor")
print(f"  Shape: {M.shape}")

In [ ]:
print(f"\nComputing causal norms for {N:,} sampled tokens...")

# Extract embeddings for sampled tokens
sampled_embeddings = gamma[token_indices]  # [N, hidden_dim]

# Compute causal norms: ||γᵢ||_M = sqrt(γᵢ^T M γᵢ)
M_gamma = sampled_embeddings @ M.to(DEVICE)  # [N, hidden_dim]
causal_norms = torch.sqrt((sampled_embeddings * M_gamma).sum(dim=-1))  # [N]
causal_norms = causal_norms.numpy()

print(f"✓ Computed causal norms")
print(f"\nCausal norm statistics:")
print(f"  Min: {causal_norms.min():.2f} logometers")
print(f"  Max: {causal_norms.max():.2f} logometers")
print(f"  Mean: {causal_norms.mean():.2f} logometers")
print(f"  Median: {np.median(causal_norms):.2f} logometers")

## Density vs Norm Scatter Plot

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

# Scatter plot colored by density
scatter = ax.scatter(
    causal_norms,
    densities,
    c=densities,
    cmap='viridis',
    alpha=0.5,
    s=10,
    rasterized=True  # Faster rendering for many points
)

plt.colorbar(scatter, ax=ax, label='Local Density (1/logometers)')

ax.set_xlabel('Causal Norm (logometers)', fontsize=12)
ax.set_ylabel('Local Density (1/logometers)', fontsize=12)
ax.set_yscale('log')
ax.set_title(f'Local Density vs Causal Norm (N={N:,}, k={K_NEIGHBORS})', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Interpretation:")
print("  - High density at small norm → Core is DENSE (tightly packed)")
print("  - Low density at small norm → Core is SPARSE (isolated tokens)")
print("  - Flat density across norms → Uniform distribution (no radial structure)")

## Radial Density Profile

Bin by causal norm and compute mean density in each bin.

In [ ]:
# Create radial bins
n_bins = 30
norm_bins = np.linspace(causal_norms.min(), causal_norms.max(), n_bins + 1)
bin_centers = (norm_bins[:-1] + norm_bins[1:]) / 2

# Compute mean density in each bin
bin_indices = np.digitize(causal_norms, norm_bins) - 1
bin_indices = np.clip(bin_indices, 0, n_bins - 1)  # Handle edge cases

mean_density_per_bin = np.zeros(n_bins)
std_density_per_bin = np.zeros(n_bins)
count_per_bin = np.zeros(n_bins)

for i in range(n_bins):
    mask = bin_indices == i
    if mask.sum() > 0:
        mean_density_per_bin[i] = densities[mask].mean()
        std_density_per_bin[i] = densities[mask].std()
        count_per_bin[i] = mask.sum()

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Top: Mean density vs norm
ax1.plot(bin_centers, mean_density_per_bin, 'o-', color='blue', linewidth=2, markersize=6)
ax1.fill_between(
    bin_centers,
    mean_density_per_bin - std_density_per_bin,
    mean_density_per_bin + std_density_per_bin,
    alpha=0.3,
    color='blue'
)
ax1.set_xlabel('Causal Norm (logometers)', fontsize=12)
ax1.set_ylabel('Mean Local Density (1/logometers)', fontsize=12)
ax1.set_title('Radial Density Profile', fontsize=14, fontweight='bold')
ax1.grid(alpha=0.3)

# Bottom: Count histogram (for context)
ax2.bar(bin_centers, count_per_bin, width=(norm_bins[1] - norm_bins[0]), alpha=0.7, color='gray', edgecolor='black')
ax2.set_xlabel('Causal Norm (logometers)', fontsize=12)
ax2.set_ylabel('Token Count', fontsize=12)
ax2.set_title('Token Count by Norm (for reference)', fontsize=13)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Key questions:")
print("  - Does density INCREASE toward the core (small norm)?")
print("  - Does density DECREASE toward the core?")
print("  - Is density roughly FLAT across all norms?")

## Statistical Analysis: Core vs Shell

In [ ]:
# Define core and shell regions based on norm distribution
median_norm = np.median(causal_norms)

# Core: bottom 10% by norm
core_threshold = np.percentile(causal_norms, 10)
core_mask = causal_norms < core_threshold

# Shell: top 50% near the peak (around median)
shell_mask = (causal_norms > np.percentile(causal_norms, 40)) & (causal_norms < np.percentile(causal_norms, 60))

# Outer: top 10%
outer_threshold = np.percentile(causal_norms, 90)
outer_mask = causal_norms > outer_threshold

print("\n" + "=" * 60)
print("DENSITY COMPARISON: Core vs Shell vs Outer")
print("=" * 60)

print(f"\nCore (norm < {core_threshold:.2f}):")
print(f"  N tokens: {core_mask.sum():,}")
print(f"  Mean density: {densities[core_mask].mean():.6f}")
print(f"  Std density: {densities[core_mask].std():.6f}")

print(f"\nShell (norm ≈ {median_norm:.2f}, middle 20%):")
print(f"  N tokens: {shell_mask.sum():,}")
print(f"  Mean density: {densities[shell_mask].mean():.6f}")
print(f"  Std density: {densities[shell_mask].std():.6f}")

print(f"\nOuter (norm > {outer_threshold:.2f}):")
print(f"  N tokens: {outer_mask.sum():,}")
print(f"  Mean density: {densities[outer_mask].mean():.6f}")
print(f"  Std density: {densities[outer_mask].std():.6f}")

# Ratio
core_shell_ratio = densities[core_mask].mean() / densities[shell_mask].mean()
print(f"\nCore/Shell density ratio: {core_shell_ratio:.2f}x")

if core_shell_ratio > 1.5:
    print("\n✅ Core is DENSER than shell (concentrated center)")
elif core_shell_ratio < 0.67:
    print("\n✅ Core is SPARSER than shell (hollow structure)")
else:
    print("\n⚠️  Core and shell have similar density (no strong radial gradient)")

## Summary

**Question:** Is the "void" at small causal norms a true void (low density) or just low count?

**Method:** k-NN density estimation on 32k token subsample

**Answer:** Check the density vs norm plot and core/shell comparison above!

---

**Possible outcomes:**
1. **High density at core:** Not a void - just rare tokens, but tightly packed when they exist
2. **Low density at core:** True void - core region is genuinely sparse
3. **Flat density:** No radial structure in density (only in count distribution)